# K-Means Anchors Ratios Tutorial

In [1]:
import json
import numpy as np
import os

%load_ext autoreload
%autoreload 2

## Get COCO train 2017 instances

In [2]:
INSTANCES_PATH = "./annotations/instances_train2017.json"

!pip install -r requirements.txt
if not os.path.exists(INSTANCES_PATH):
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip annotations_trainval2017.zip

In [3]:
with open(INSTANCES_PATH) as f:
    instances = json.load(f)

## Get optimal anchors ratios

In [4]:
## change the following parameters according to your model:

# EfficientDetD{PHI}
PHI = 0  # for another efficientdet change only this, e.g. PHI = 3 for D3

input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
pyramid_levels = [5, 5, 5, 5, 5, 5, 5, 5, 6]
anchor_scale = [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0]

scale = anchor_scale[PHI]
strides = 2 ** np.arange(3, pyramid_levels[PHI] + 3)
scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

INPUT_SIZE = input_sizes[PHI]
ANCHORS_SIZES = (scale * scales * strides[:, np.newaxis]).flatten().tolist()
ANCHORS_SIZES

[32.0,
 40.31747359663594,
 50.79683366298238,
 64.0,
 80.63494719327188,
 101.59366732596476,
 128.0,
 161.26989438654377,
 203.18733465192952,
 256.0,
 322.53978877308754,
 406.37466930385904,
 512.0,
 645.0795775461751,
 812.7493386077181]

In [5]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios

anchors_ratios = get_optimal_anchors_ratios(
    instances,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    normalizes_bboxes=True,
    num_runs=3,
    num_anchors_ratios=3,
    max_iter=300,
    iou_threshold=0.5,
    min_size=0,
    decimals=1,
    default_anchors_ratios=[(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
)

[10/28 18:31:22] Starting the calculation of the optimal anchors ratios
[10/28 18:31:22] Extracting and preprocessing bounding boxes
[10/28 18:31:24] Discarding 2 bounding boxes with size lower or equal to 0


[10/28 18:31:24] K-Means (3 runs): 100%|██████████████████| 3/3 [00:27<00:00,  9.17s/it]

	Runs avg. IoU: 80.48% ± 0.00% (mean ± std. dev. of 3 runs, 0 skipped)
	Avg. IoU between bboxes and their most similar anchors after norm. them to make their area equal (only ratios matter): 80.48%


[10/28 18:31:56] Default anchors ratios: [(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar default anchors, no norm. (both ratios and sizes matter): 62.25%
	Num. bboxes without similar default anchors (IoU < 0.5):  250598/860001 (29.14%)
[10/28 18:32:01] K-Means anchors ratios: [(0.6, 1.5), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar K-Means anchors, no norm. (both ratios and sizes matter): 62.63%
	Num. bboxes without similar K-Means anchors (IoU < 0.5):  239153/860001 (27.81%)
[10/28 18:32:03] K-Means anchors have an IoU < 50% with bboxes in 1.33% less cases than the default anchors, you should consider to use them


## Generate anchors given ratios and sizes

In [6]:
from kmeans_anchors_ratios import generate_anchors_given_ratios_and_sizes


anchors = generate_anchors_given_ratios_and_sizes(anchors_ratios, ANCHORS_SIZES)
print("Anchors:")
print(anchors)

Anchors:
[[  19.2          48.        ]
 [  32.           32.        ]
 [  44.8          22.4       ]
 [  24.19048416   60.47621039]
 [  40.3174736    40.3174736 ]
 [  56.44446304   28.22223152]
 [  30.4781002    76.19525049]
 [  50.79683366   50.79683366]
 [  71.11556713   35.55778356]
 [  38.4          96.        ]
 [  64.           64.        ]
 [  89.6          44.8       ]
 [  48.38096832  120.95242079]
 [  80.63494719   80.63494719]
 [ 112.88892607   56.44446304]
 [  60.9562004   152.39050099]
 [ 101.59366733  101.59366733]
 [ 142.23113426   71.11556713]
 [  76.8         192.        ]
 [ 128.          128.        ]
 [ 179.2          89.6       ]
 [  96.76193663  241.90484158]
 [ 161.26989439  161.26989439]
 [ 225.77785214  112.88892607]
 [ 121.91240079  304.78100198]
 [ 203.18733465  203.18733465]
 [ 284.46226851  142.23113426]
 [ 153.6         384.        ]
 [ 256.          256.        ]
 [ 358.4         179.2       ]
 [ 193.52387326  483.80968316]
 [ 322.53978877  322.53978877]

## Get bounding boxes adapted to the input size

In [7]:
from kmeans_anchors_ratios import get_bboxes_adapted_to_input_size


resized_bboxes = get_bboxes_adapted_to_input_size(instances, input_size=INPUT_SIZE)
resized_bboxes = resized_bboxes[resized_bboxes.prod(axis=1) > 0]  # remove 0 size
print("Bounding boxes adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes adapted to the input size (first 5):
[[ 62.168  56.704]
 [119.512  30.84 ]
 [128.     46.248]
 [100.752  18.168]
 [ 53.176  53.4  ]]


## Get the avg. IoU between the bounding boxes and their closest anchors

In [8]:
from kmeans_anchors_ratios import average_iou


avg_iou = average_iou(resized_bboxes, anchors)
print(f"Avg. IoU: {100 * avg_iou:.2f}%")

Avg. IoU: 62.63%


## Get annotations whose bounding boxes don't have similar anchors

In [9]:
from kmeans_anchors_ratios import get_annotations_without_similar_anchors


annotations = get_annotations_without_similar_anchors(
    instances,
    anchors_ratios,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    iou_threshold=0.5,
    min_size=0,
)

bboxes = [ann["bbox"][-2:] for ann in annotations]  # widths and heights
print("Bounding boxes without similar anchors (first 5):")
print(bboxes[:5])

instances_without_similar_anchors = instances.copy()
instances_without_similar_anchors["annotations"] = annotations
resized_bboxes = get_bboxes_adapted_to_input_size(instances_without_similar_anchors, input_size=INPUT_SIZE)
print("Bounding boxes without similar anchors adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes without similar anchors (first 5):
[[125.94, 22.71], [26.17, 19.49], [13.28, 9.36], [34.03, 22.54], [29.63, 25.24]]
Bounding boxes without similar anchors adapted to the input size (first 5):
[[100.752  18.168]
 [ 20.936  15.592]
 [ 10.624   7.488]
 [ 27.224  18.032]
 [ 23.704  20.192]]
